##### Объединение всех фичей в одну таблицу

In [1]:
pre = '../../' # путь до главной директории проекта
%run ../init.ipynb #подгрузить нужные библиотеки

In [2]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/feature_combine.xlsx"
writer = ExcelWriter(name)

In [3]:
name = pre + "data/interm/target_data.pkl"
target_data = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_data))
# target_data.head(1)

127968


In [4]:
# список договоров

name = pre + "data/interm/lst_contracts.pkl"
lst_contracts = pd.read_pickle(name, compression={'method':'zip'})

print(len(lst_contracts))
# lst_contracts.head(1)

279643


In [5]:
# фича okved  - ОКВЭД

name = pre + "data/interm/feature_okved.pkl"
target_okved = pd.read_pickle(name, compression={'method':'zip'})

num, duplicates = identify_duplicates(target_okved, 'PL')
Check_dupl(num, duplicates)

print(len(target_okved))
# target_okved.head(1)

127968


In [6]:
#  наличия просрочки по договору на определенную дату для раcчета фичи contract_ovd_cnt

name = pre + "data/interm/overdue_contracts.pkl"
overdue_contracts = pd.read_pickle(name, compression={'method':'zip'})

print(len(overdue_contracts))
# overdue_contracts.head(1)

207598


In [7]:
# показатель max_leas_ovd_pay_dt - дата последней оплаченной просроченной задолженности по лизинговым платежам

name = pre + "data/interm/leas_ovd_pay_dt.pkl"
leas_ovd_pay_dt = pd.read_pickle(name, compression={'method':'zip'})

print(len(leas_ovd_pay_dt))
# leas_ovd_pay_dt.head(1)

101634


In [8]:
keep_col = ['ИНН контрагента', 'decision_date', 'CONTRACT_KEY', 'CONTRACT_APP_KEY', 'CONTRACT_NUM', 'CLOSE_DT',	'open_auxilary', 'LEASE_DATE']
target_contr = pd.merge(target_data, lst_contracts[keep_col], how='left', on=['ИНН контрагента', 'decision_date'])

keep_col = ['decision_date', 'CONTRACT_KEY', 'OVERDUE_FLG']
target_contr_over = pd.merge(target_contr, overdue_contracts[keep_col], how='left', on=['CONTRACT_KEY', 'decision_date'])

keep_col = ['decision_date', 'CONTRACT_KEY', 'max_leas_ovd_pay_dt']
target_param = pd.merge(target_contr_over, leas_ovd_pay_dt[keep_col], how='left', on=['CONTRACT_KEY', 'decision_date'])
# target_param.head(1)

In [9]:
def summary(x):
    tmp = x[x['OVERDUE_FLG']==1]
    result = {
        'contract_cnt': x['CONTRACT_NUM'].nunique(),
        'max_leas_ovd_pay_dt': x['max_leas_ovd_pay_dt'].max(),
        'contract_ovd_cnt':  tmp['CONTRACT_NUM'].nunique()
    }
    return pd.Series(result)

In [10]:
# слишком долго
# features = target_param.groupby(['PL']).apply(summary)
# features.head(1)

In [11]:
df_ovd_pay = target_param.groupby(['PL']).agg({'CONTRACT_NUM':'nunique', 'max_leas_ovd_pay_dt':'max'}).reset_index()

df_ovd_pay.rename(columns={'CONTRACT_NUM':'contract_cnt'}, inplace=True)

In [12]:
# agg работает намного быстрее

# без тех, что закончились более 6 месяцев назад
mask_closed = (target_param['Дата принятия решения по сделке']<=target_param['CLOSE_DT']+timedelta(days=180)) # 180 полгода
target_6 = target_param[mask_closed]
df_cnt_6 = target_6.groupby(['PL']).agg({'CONTRACT_NUM':'nunique'}).reset_index()
df_cnt_6.rename(columns={'CONTRACT_NUM':'ctr_count_6m'}, inplace=True)

df_ovd_pay = target_param.groupby(['PL']).agg({'CONTRACT_NUM':'nunique', 'max_leas_ovd_pay_dt':'max'}).reset_index()
df_ovd_pay.rename(columns={'CONTRACT_NUM':'contract_cnt'}, inplace=True)

target_overdue_flg = target_param[target_param['OVERDUE_FLG']==1]
df_ovd = target_overdue_flg.groupby(['PL']).agg({'CONTRACT_NUM':'nunique'}).reset_index()
df_ovd.rename(columns={'CONTRACT_NUM':'contract_ovd_cnt'}, inplace=True)

In [13]:
df = pd.merge(target_data, target_okved, how='left', on='PL')
df = pd.merge(df, df_cnt_6, how='left', on='PL')
df = pd.merge(df, df_ovd_pay, how='left', on='PL')
target_features= pd.merge(df, df_ovd, how='left', on='PL')

Check_length(len(target_data), len(target_features))

target_features['dsnc_ovd_pay_days'] = (pd.to_datetime(target_features['decision_date']) \
                            - pd.to_datetime(target_features['max_leas_ovd_pay_dt'])).dt.days

target_features['contract_ovd_cnt'] = target_features['contract_ovd_cnt'].fillna(0)
target_features['contract_cnt'] = target_features['contract_cnt'].fillna(0)
target_features['ctr_count_6m'] = target_features['ctr_count_6m'].fillna(0)

for col in ['contract_ovd_cnt', 'contract_cnt', 'ctr_count_6m']:
    target_features[col] = target_features[col].astype(int)
    
# target_features.head(3)

In [14]:
print(len(target_features))
target_features.dtypes

127968


PL                                         object
Номер сделки                               object
Номер договора (кратко)                    object
ИНН контрагента                            object
Продукт                                    object
Дата принятия решения по сделке    datetime64[ns]
decision_date                      datetime64[ns]
Метка кред дефолта                          int64
okved                                      object
ctr_count_6m                                int32
contract_cnt                                int32
max_leas_ovd_pay_dt                datetime64[ns]
contract_ovd_cnt                            int32
dsnc_ovd_pay_days                         float64
dtype: object

In [15]:
target_features.describe()

,Метка кред дефолта,ctr_count_6m,contract_cnt,contract_ovd_cnt,dsnc_ovd_pay_days
count,127968.000000,127968.000000,127968.000000,127968.000000,34115.000000
mean,0.037830,1.180334,1.622484,0.270247,229.835087
std,0.190785,3.020038,4.318072,1.279713,317.085769
min,0.000000,0.000000,0.000000,0.000000,2.000000
25%,0.000000,0.000000,0.000000,0.000000,29.000000
50%,0.000000,0.000000,0.000000,0.000000,98.000000
75%,0.000000,1.000000,2.000000,0.000000,298.000000
max,1.000000,260.000000,260.000000,49.000000,2662.000000


In [16]:
max_ctr_count_6m = target_features['ctr_count_6m'].max()
max_contract_ovd_cnt = target_features['contract_ovd_cnt'].max()

limit_ctr_count_6m = np.percentile(target_features['ctr_count_6m'], 99)
limit_contract_ovd_cnt = np.percentile(target_features['contract_ovd_cnt'], 99)

print("Ограничиваем ctr_count_6m не более {}  при макс значении {}".format(limit_ctr_count_6m, max_ctr_count_6m))
print("Ограничиваем contract_ovd_cnt не более {}  при макс значении {}".format(limit_contract_ovd_cnt, max_contract_ovd_cnt))

target_features_limit = target_features[(target_features['ctr_count_6m'] <= limit_ctr_count_6m) & (target_features['contract_ovd_cnt'] <= limit_contract_ovd_cnt)]

print("Изменение выборки с {} до {}".format(len(target_features), len(target_features_limit)))

Ограничиваем ctr_count_6m не более 12.0  при макс значении 260
Ограничиваем contract_ovd_cnt не более 5.0  при макс значении 49
Изменение выборки с 127968 до 125908


In [17]:
target_features_res = target_features_limit.copy()

target_features_res.drop(columns='Дата принятия решения по сделке', inplace=True)
target_features_res.rename(columns={'decision_date':'Дата принятия решения по сделке'}, inplace=True)
target_features_res.rename(columns={'dsnc_ovd_pay_days':'day_from_last_paid_delay'}, inplace=True)
target_features_res.rename(columns={'contract_ovd_cnt':'debt_ctr'}, inplace=True)
target_features_res.rename(columns={'okved':'lessee_okved_1'}, inplace=True)
 
target_features_res.to_pickle(pre+"data/interm/target_features.pkl", compression={'method': 'zip'})
 
keep_col = ['PL', 'Номер сделки', 'Номер договора (кратко)', 'ИНН контрагента', 'Продукт',
                'Дата принятия решения по сделке', 
                'Метка кред дефолта', 
                'lessee_okved_1', 'ctr_count_6m', 'debt_ctr',	'day_from_last_paid_delay']

target_features_res[keep_col].to_csv(pre+"data/processed/target_features.csv", index=False, sep=";", encoding="1251", date_format='%Y.%m.%d')
# target_features_res[keep_col].head(1)

In [18]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save() 